In [1]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np

hubs_dict = {'ARICA': 'IQUIQUE',
 'IQUIQUE': 'IQUIQUE',
 'ANTOFAGASTA': 'ANTOFAGASTA',
 'COPIAPO': 'COPIAPO',
 'COQUIMBO': 'COQUIMBO',
 'OVALLE': 'COQUIMBO',
 'ILLAPEL': 'CURAUMA',
 'LLAY LLAY': 'CURAUMA',
 'CURAUMA': 'CURAUMA',
 'CERVECERA': 'SANTIAGO SUR',
 'MODELO': 'SANTIAGO SUR',
 'SANTIAGO SUR': 'SANTIAGO SUR',
 'RANCAGUA': 'SANTIAGO SUR',
 'TALCA': 'TALCA',
 'CHILLAN': 'TALCAHUANO',
 'TALCAHUANO': 'TALCAHUANO',
 'LOS ANGELES': 'TALCAHUANO',
 'TEMUCO': 'TEMUCO',
 'VALDIVIA': 'VALDIVIA',
 'OSORNO': 'VALDIVIA',
 'PUERTO MONTT': 'PUERTO MONTT',
 'CASTRO': 'PUERTO MONTT',
 'COYHAIQUE': 'COYHAIQUE',
 'CALAMA': 'ANTOFAGASTA'}

sheets_dict1 = pd.read_excel('../data/venta SKU por CD 2018.xlsx', sheet_name=None)
sheets_dict2 = pd.read_excel('../data/venta SKU por CD 2019.xlsx', sheet_name=None)
sheets_dict3 = pd.read_excel('../data/venta SKU por CD 2020.xlsx', sheet_name=None)

master = pd.DataFrame()
for name in sheets_dict1:
    
    df = sheets_dict1[name]
    master = pd.concat([master, df])
    
for name in sheets_dict2:
    
    df = sheets_dict2[name]
    master = pd.concat([master, df])
    
for name in sheets_dict3:
    
    df = sheets_dict3[name]
    master = pd.concat([master, df])
    
print('MASTER ARMADA')

master = master.drop(columns='DESCR_PROD')

datos_agrupados = master.groupby(by=['DESCR_CENDIST', 'ID_SKU_VENTA', 'FECHA']).sum().reset_index()

with open('../data/skus_filter.json', 'r') as file:
    skus_filter = set(json.load(file))
    
cds_validos = set(hubs_dict.keys())

datos_agrupados = datos_agrupados[(~datos_agrupados['ID_SKU_VENTA'].isin(skus_filter)) & (datos_agrupados['DESCR_CENDIST'].isin(cds_validos))]

print('FILTROS APLICADOS')

def date_from_string(string):
    lista = string.split('/')
    if lista[2] == '18':
        return 2018, int(lista[1]), int(lista[0])
    elif lista[2] == '19':
        return 2019, int(lista[1]), int(lista[0])
    elif lista[2] == '20':
        return 2020, int(lista[1]), int(lista[0])
    
datos_agrupados['FECHA'] = datos_agrupados.apply(lambda x: date(*date_from_string(x.FECHA)), axis=1)

print('FORMATO FECHA LISTO')

cdList, skuList, fechasList, zeros = [], [], [], []
for cd in datos_agrupados['DESCR_CENDIST'].unique():
    for sku in datos_agrupados['ID_SKU_VENTA'].unique():
        for fecha in datos_agrupados['FECHA'].unique():
            cdList.append(cd)
            skuList.append(sku)
            fechasList.append(fecha)
            zeros.append(0)
            
print('DATAFRAME AUXILIAR LISTO')
            
aux_dict = {'DESCR_CENDIST': cdList, 'ID_SKU_VENTA': skuList, 'FECHA': fechasList, 'Venta en cajas': zeros}
aux_dataframe = pd.DataFrame.from_dict(aux_dict)
datos_agrupados = pd.concat([datos_agrupados, aux_dataframe])
datos_agrupados_fill_zeros = datos_agrupados.groupby(by=['DESCR_CENDIST', 'ID_SKU_VENTA', 'FECHA']).sum().reset_index()

print('FILL ZEROS LISTO')

conversion = pd.read_csv('../data/conversiones_unidades.csv', encoding = "ISO-8859-1")[['ID_SKU_VENTA', 'CAJAS_POR_PALLET']]
sku_pallets = {}
for index, row in conversion.iterrows():
    sku_pallets[row['ID_SKU_VENTA']] = row['CAJAS_POR_PALLET']
    
datos_agrupados_fill_zeros['Venta en cajas'] = datos_agrupados_fill_zeros.apply(lambda x: x['Venta en cajas'] / sku_pallets[x['ID_SKU_VENTA']], axis=1)
datos_agrupados_fill_zeros = datos_agrupados_fill_zeros.rename(columns={'Venta en cajas': 'Venta en pallets'})

print('CONVERSION A PALLETS LISTA')

datos_agrupados_fill_zeros = datos_agrupados_fill_zeros.sort_values(by=['DESCR_CENDIST', 'ID_SKU_VENTA', 'FECHA'])

MASTER ARMADA
FILTROS APLICADOS
FORMATO FECHA LISTO
DATAFRAME AUXILIAR LISTO
FILL ZEROS LISTO
CONVERSION A PALLETS LISTA


In [3]:
datos_agrupados_fill_zeros = datos_agrupados_fill_zeros.drop(columns='VENTA_CAJAS')

In [4]:
datos_agrupados_fill_zeros

,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en pallets
0,ANTOFAGASTA,515,2018-01-02,0.100
1,ANTOFAGASTA,515,2018-01-03,0.125
2,ANTOFAGASTA,515,2018-01-04,0.275
3,ANTOFAGASTA,515,2018-01-05,0.400
4,ANTOFAGASTA,515,2018-01-06,0.000
...,...,...,...,...
34285051,VALDIVIA,875556,2020-09-25,0.000
34285052,VALDIVIA,875556,2020-09-26,0.000
34285053,VALDIVIA,875556,2020-09-28,0.000
34285054,VALDIVIA,875556,2020-09-29,0.000


In [6]:
datos_agrupados_fill_zeros = datos_agrupados_fill_zeros.astype({'FECHA': 'datetime64[ns]'})

In [8]:
datos_agrupados_fill_zeros.to_csv('../data/datos_limpios.csv')

In [7]:
import pantab
datos_agrupados_fill_zeros = datos_agrupados_fill_zeros.dropna()
pantab.frame_to_hyper(datos_agrupados_fill_zeros, "../data/datos_limpios.hyper", table="evalpro")

In [9]:
check = pd.read_csv('../data/datos_limpios.csv')

In [10]:
check

,Unnamed: 0,DESCR_CENDIST,ID_SKU_VENTA,FECHA,Venta en pallets
0,0,ANTOFAGASTA,515,2018-01-02,0.100
1,1,ANTOFAGASTA,515,2018-01-03,0.125
2,2,ANTOFAGASTA,515,2018-01-04,0.275
3,3,ANTOFAGASTA,515,2018-01-05,0.400
4,4,ANTOFAGASTA,515,2018-01-06,0.000
...,...,...,...,...,...
34285051,34285051,VALDIVIA,875556,2020-09-25,0.000
34285052,34285052,VALDIVIA,875556,2020-09-26,0.000
34285053,34285053,VALDIVIA,875556,2020-09-28,0.000
34285054,34285054,VALDIVIA,875556,2020-09-29,0.000
